# Introducción a Polars 🐻‍❄️

> **Descripción:** Cuaderno de contenidos sobre introducción a Polars para el Bootcamp de Ciencia de Datos en Código Facilito, 2023. <br>
> **Autor:** [Rodolfo Ferro](https://github.com/RodolfoFerro) <br>
> **Contacto:** [Twitter](https://twitter.com/rodo_ferro) / [Instagram](https://www.instagram.com/rodo_ferro/)


## Contenido

### **Sección I**
- ¿Qué es Polars?
- Polars vs. Pandas
- El crecimiento de Polars

### **Sección II**
- Tipos y estructuras de datos
- Contextos y expresiones
- Lazy / Eager API

### **Sección III – Ejercicios**
- Ejemplos con expresiones y transformaciones
- SQL context
- Ejercicios de tarea


## **Sección I**

Para más detalles, te recomiendo revisar la presentación que puedes encontrar [aquí](https://rodolfoferro.xyz/polars-facilito/).

### **¿Qué es Polars?**

<center>
    <img src="https://raw.githubusercontent.com/pola-rs/polars-static/master/logos/polars_github_logo_rect_dark_name.svg" width="70%">
</center>

**Polars** es una _**DataFrame** library_ de código abierto y de alto rendimiento para manipular datos estructurados. Su core está escrito en Rust, pero la biblioteca está disponible en Python, Rust y NodeJS.

### **Polars vs Pandas**

Si bien, **Pandas** es una de las bibliotecas más utilizadas para trabajar con datos, una ventaja de **Polars**, al estar hecho con un lenguaje compilado, es que le permite tener un alto rendimiento para manipular datos estructurados.

Hay un benchmark realizado por el equipo de Polars, el cual puedes revisar aquí: https://www.pola.rs/benchmarks.html

En el link anterior podrás encontrar gráficos como estos:

<center>
    <img src="https://raw.githubusercontent.com/pola-rs/polars-static/master/benchmarks/tpch/sf_10_and_io.png" width="70%">
    <img src="https://raw.githubusercontent.com/pola-rs/polars-static/master/benchmarks/tpch/sf_10.png" width="70%">
</center>

Estos gráficos muestran resultados de pruebas de rendimiento al trabajar Polars y en contraste con otras herremientas (en elleas incluido Pandas). Dichos benchmarks son básicamente pruebas de memoria y carga de datos.

Puedes obtener más detalles de dicho benchmark en el link ya meniconado.

## **Sección II**

Comenzamos con la instalación de `polars`:

In [ ]:
!pip install polars

### **Tipos y estructuras de datos**

En esta sección, exploraremos en detalle los tipos y estructuras de datos que Polars ofrece como alternativa a Pandas. Una comprensión sólida de estos elementos es esencial para aprovechar al máximo las capacidades de Polars y tomar decisiones informadas sobre cuándo y cómo usar esta librería en lugar de otras opciones como Pandas.

Todos los tipos y las estructuras de datos están basadas en `Arrow`, una implementación completa, segura y nativa de Rust de [_Apache Arrow_](https://arrow.apache.org/), que es una plataforma de desarrollo multilenguaje para datos en memoria.

#### Tipos de Datos en Polars



Polars introduce una gama de tipos de datos optimizados que permiten un mejor rendimiento y uso eficiente de la memoria en comparación con Pandas. Algunos de los tipos de datos clave en Polars incluyen:

- **Integer:** Polars ofrece varios tipos de enteros con diferentes tamaños, como `Int8`, `Int16`, `Int32` y `Int64`. Asimismo, números enteros sin signo, como `UInt8`, `UInt16`, `UInt32` y `UInt64`. Estos tipos permiten un control más preciso sobre la cantidad de memoria utilizada.

- **Floating-Point:** Al igual que Pandas, Polars ofrece tipos de punto flotante como `Float32` y `Float64` para manejar números decimales con diferentes niveles de precisión.

- **Boolean:** Polars utiliza el tipo `Boolean` para representar valores booleanos (verdadero/falso) de manera eficiente.

- **Temporal:** Polars proporciona tipos de datos para manejar fechas y horas, como `Date` y `Datetime`, lo que facilita el trabajo con datos temporales.

#### Estructuras de Datos en Polars

Polars introduce dos estructuras de datos principales: `DataFrame` y `Series`, que son equivalentes a las estructuras homónimas en Pandas:

- **DataFrame:** El equivalente a un DataFrame en Polars es una estructura tabular que organiza los datos en filas y columnas. Polars ofrece una forma de crear y manipular DataFrames, lo que permite realizar operaciones complejas de manera eficiente.

- **Series:** Las Series son equivalentes a columnas en un DataFrame. Pueden contener un solo tipo de dato y se utilizan para realizar operaciones vectorizadas en los datos.

> **Nota:** Polars introduce los LazyFrames. Esencialmente, un `LazyFrame` es una forma más eficiente de trabajar con un conjunto de datos que usar DataFrame. Si reemplazas tu DataFrame con LazyFrame en tu código con Polars, puedes obtener un tiempo de ejecución más rápido.

In [ ]:
import polars as pl

s = pl.Series("a", [1, 2, 3, 4, 5])
print(s)

In [ ]:
from datetime import datetime

df = pl.DataFrame(
    {
        "integer": [1, 2, 3, 4, 5],
        "date": [
            datetime(2022, 1, 1),
            datetime(2022, 1, 2),
            datetime(2022, 1, 3),
            datetime(2022, 1, 4),
            datetime(2022, 1, 5),
        ],
        "float": [4.0, 5.0, 6.0, 7.0, 8.0],
    }
)

print(df)

### **Contextos y expresiones**

Polars ha desarrollado su propio lenguaje específico de dominio (DSL) para transformar datos. El lenguaje es muy fácil de usar y permite consultas complejas que siguen siendo legibles por humanos. Los dos componentes centrales del lenguaje son "Contextos" y "Expresiones".

#### Contextos

Un contexto, como lo implica el nombre, se refiere al contexto en el que se debe evaluar una expresión. Hay tres contextos principales:

- Selección: `df.select([..])`, `df.with_columns([..])`
- Filtrado: `df.filter()`
- Agrupaciones y agregaciones: `df.groupby(..).agg([..])`

Revisemos algunos ejemplos.


Comenzemos creando un nuevo dataframe con algo de información.

In [ ]:
import numpy as np

# Creamos un dataframe para trabajar con él
df = pl.DataFrame(
    {
        "nid": [1, 2, 3, None, 5],
        "names": ["Rodo", "Hiram", "Josué", "David", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "C", "B", "B"],
    }
)
print(df)

##### Contexto `select`

En este contexto, la selección aplica expresiones sobre columnas. Las expresiones en este contexto deben producir series que tengan la misma longitud o una longitud de 1.

Una selección puede producir nuevas columnas que son agregaciones, combinaciones de expresiones o literales.

In [ ]:
out = df.select(
    pl.sum("nid"),
    #pl.col("names").sort(),
    #pl.col("names").first().alias("first name"),
    #(pl.mean("nid") * 10).alias("10xnid"),
)
print(out)

El contexto de selección es muy poderoso y nos permite realizar expresiones arbitrarias independientes (y en paralelo) entre sí.

De manera similar a `select`, existe la sentencia `with_columns` que también es una entrada al contexto de selección. La principal diferencia es que `with_columns` conserva las columnas originales y agrega otras nuevas, mientras que `select` elimina las columnas originales.

In [ ]:
df = df.with_columns(
    pl.sum("nid").alias("nid_sum"),
    #pl.col("random").count().alias("count"),
)
print(df)

##### Contexto `filter`

En este contexto, se filtra el marco de datos existente en función de la expresión arbitraria que se evalúa como el tipo de datos booleano.

In [ ]:
out = df.filter(pl.col("random") > 0.5)
print(out)

##### Contexto `groupby` / `aggregation`

En este contexto, las expresiones funcionan en grupos, por lo que pueden producir resultados de cualquier longitud (un grupo puede tener muchos miembros).

In [ ]:
out = df.groupby("groups").agg(
    pl.sum("nid"),  # Suma los nid por groupos
    #pl.col("random").count().alias("count"),  # Cuenta miembros de grupo
    # Suma random cuando name != null
    #pl.col("random").filter(pl.col("names").is_not_null()).sum().suffix("_sum"),
    #pl.col("names").reverse().alias("reversed names"),
)
print(out)


#### Expresiones

Polars cuenta con expresiones. Las expresiones son el núcleo de muchas operaciones de ciencia de datos y son el concepto fundamental de Polars para su rendimiento muy rápido.

Algunas de estas operaciones importantes en la ciencia de datos son:

- tomar una muestra de filas de una columna
- multiplicar valores en una columna
- extraer una columna de años a partir de fechas
- convertir una columna de cadenas a minúsculas
- ¡y más!

Sin embargo, las expresiones también se utilizan dentro de otras operaciones:

- tomar la media de un grupo en una operación `groupby`
- calcular el tamaño de los grupos en una operación `groupby`
- tomando la suma horizontalmente a través de las columnas

Polars realiza estas transformaciones de datos centrales muy rápidamente con:

- optimización automática de consultas en cada expresión
- paralelización automática de expresiones en muchas columnas

**Analicemos.** ¿Qué hace la siguiente sentencia?

In [ ]:
out = (pl.col("random").sort() > 0.5).suffix("_condition")
print(out)

Notemos que al ejecutar no obtenemos un resultado, esto es porque es necesario ejecutar estas sentencias dentro de un contexto. **Veamos.**

In [ ]:
out = df.select((pl.col("random").sort() > 0.5).suffix("_condition"))
print(out)

In [ ]:
out = df.with_columns((pl.col("random").sort() > 0.5).suffix("_condition"))
print(out)

**Observación:** ¿Por qué si ejecutamos un sort, los datos no están ordenados?

### **Operaciones "Lazy" y "Eager"**

Una característica única de Polars es su enfoque en las operaciones "Lazy" y "Eager". Las operaciones "Lazy" permiten construir una secuencia de operaciones en un DataFrame sin ejecutarlas de inmediato. Esto puede ser útil para optimizar el rendimiento y evitar cálculos innecesarios. Por otro lado, las operaciones "Eager" ejecutan inmediatamente las operaciones en el DataFrame y devuelven los resultados.

**Analicemos.** ¿Qué sucede en el siguiente ejemplo?

In [ ]:
%time
df = pl.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv")
df_small = df.filter(pl.col("sepal_length") > 5)
df_agg = df_small.groupby("species").agg(pl.col("sepal_width").mean())
print(df_agg)

En este ejemplo, usamos la API "Eager" para:

- Leer el conjunto de datos del iris.
- Filtrar el conjunto de datos según la longitud del sépalo
- Calcular la media del ancho del sépalo por especie

Cada paso se ejecuta inmediatamente devolviendo los resultados intermedios. Esto puede ser un fallo a la eficiencia, ya que podríamos trabajar o cargar datos adicionales que no se están utilizando.

In [ ]:
!wget https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv

In [ ]:
%time
q = (
    pl.scan_csv("iris.csv")
    #.filter(pl.col("sepal_length") > 5)
    #.groupby("species")
    #.agg(pl.col("sepal_width").mean())
)

df = q.collect()

In [ ]:
print(df)

## **Sección III**

Estaremos poniendo en práctica lo aprendido con algunos ejercicios de [101 Pandas Exercises for Data Analysis](https://www.machinelearningplus.com/python/101-pandas-exercises-python/).

### **Ejemplos con expresiones y transformaciones**

#### 4. ¿Cómo combinar múltiples dfs en un DataFrame?

In [ ]:
# 4. ¿Cómo combinar múltiples dfs en un DataFrame?
df1 = pl.DataFrame({"letras": list("abcedfghijklmnopqrstuvwxyz")})
df2 = pl.DataFrame({"nums": np.arange(len(df1))})

print(df1, df2)

In [ ]:
### Solución


#### 14. ¿Cómo extraer items de un DataFrame dadas las posiciones a través de enteros?

In [ ]:
# 14. ¿Cómo extraer items de un DataFrame dadas las posiciones a través de enteros?
df = pl.DataFrame(list('abcdefghijklmnopqrstuvwxyz'))
pos = [0, 4, 8, 14, 20]

print(df)

In [ ]:
### Solución


#### 19. ¿Cómo calcular el número de caracteres de cada palabra en un DataFrame?

In [ ]:
# 19. ¿Cómo calcular el número de caracteres de cada palabra en un DataFrame?
df = pl.DataFrame({"palabras": ["esta", "es", "una", "palabra"]})
print(df)

In [ ]:
### Solución


#### 23. ¿Cómo convertir una cadena año-mes a fechas que comiencen en el 11 de cada mes?

In [ ]:
# 23. ¿Cómo convertir una cadena año-mes a fechas que comiencen en el 11 de cada mes?
df = pl.DataFrame(['Jan 2010', 'Feb 2011', 'Mar 2012'])
print(df)

In [ ]:
### Solución
from dateutil.parser import parse


#### 40. ¿Cómo revisar si un DataFrame tiene valores faltantes?

In [ ]:
# 40. ¿Cómo revisar si un DataFrame tiene valores faltantes?
df = pl.read_csv("https://raw.githubusercontent.com/selva86/datasets/master/Cars93_miss.csv")
print(df.head(5))

In [ ]:
### Solución
print(df.columns)


#### 49. ¿Cómo filtrar cada n-ésima fila en un DataFrame?

In [ ]:
# 49. ¿Cómo filtrar cada n-ésima fila en un DataFrame?
df = pl.read_csv("https://raw.githubusercontent.com/selva86/datasets/master/Cars93_miss.csv")
print(df.head(8))

In [ ]:
### Solución


### **SQL context**

Aunque Polars admite la escritura de consultas en SQL, se recomienda que las y los usuarios se familiaricen con la sintaxis nativa para obtener un código más legible y expresivo.

Sin embargo, si ya cuentas con una base de código SQL existente o prefieres usar SQL, Polars también te ofrece soporte para consultas SQL.

Polars utiliza el `SQLContext` para administrar consultas SQL. El contexto contiene un diccionario que asigna nombres de DataFrames y LazyFrames a sus correspondientes conjuntos de datos.

In [ ]:
# For local files use scan_csv instead
pokemon = pl.read_csv(
    "https://gist.githubusercontent.com/ritchie46/cac6b337ea52281aa23c049250a4ff03/raw/89a957ff3919d90e6ef2d34235e6bf22304f3366/pokemon.csv"
)

ctx = pl.SQLContext(register_globals=True, eager_execution=True)
df_small = ctx.execute("SELECT * from pokemon LIMIT 5")
print(df_small)

> **Para resolver la tarea, el reto es:** Poner en práctica los conocimientos adquiridos a través de ejercicios y retos.

**Puedes explorar:**
- [Polars API Reference](https://pola-rs.github.io/polars/py-polars/html/reference/index.html)
- [101 Pandas Exercises for Data Analysis](https://www.machinelearningplus.com/python/101-pandas-exercises-python/)

--------

> Contenido creado por **Rodolfo Ferro**, 2023. <br>
> Puedes contactarme a través de Insta ([@rodo_ferro](https://www.instagram.com/rodo_ferro/)) o Twitter ([@rodo_ferro](https://twitter.com/rodo_ferro)).